In [1]:
import pandas as pd
import numpy as np
import gc

import warnings
warnings.filterwarnings('ignore') #Supress unnecessary warnings for readability and cleaner presentation

In [2]:
datadir = '/data/Instacart/'

In [3]:
products_info = pd.read_csv(datadir + 'products.csv')
products = pd.read_csv(datadir + 'products_summary.csv')

products_info.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
products = pd.merge(products, products_info, on = 'product_id', how= 'left')
del products_info

In [5]:
aisles = products.groupby('aisle_id').agg({'order_count':'sum', 'reordered_order_count':'sum'}).reset_index()
aisles['aisle_reorder_ratio'] = aisles['reordered_order_count']/aisles['order_count']

aisles.head()

,aisle_id,reordered_order_count,order_count,aisle_reorder_ratio
0,1,42912,71928,0.596597
1,2,40365,82491,0.489326
2,3,272922,456386,0.598007
3,4,98243,200687,0.489533
4,5,17542,62510,0.280627


In [6]:
dept = products.groupby('department_id').agg({'order_count':'sum', 'reordered_order_count':'sum'}).reset_index()
dept['dept_reorder_ratio'] = dept['reordered_order_count']/dept['order_count']

dept.head()

,department_id,reordered_order_count,order_count,dept_reorder_ratio
0,1,1211890,2236432,0.541885
1,2,14806,36291,0.407980
2,3,739188,1176787,0.628141
3,4,6160710,9479291,0.649913
4,5,87595,153696,0.569924


In [7]:
products = pd.merge(products, aisles[['aisle_id', 'aisle_reorder_ratio']], on = 'aisle_id', how = 'left')
products = pd.merge(products, dept[['department_id', 'dept_reorder_ratio']], on = 'department_id', how = 'left')

products.head()

,product_id,user_count,weights,order_count,reordered_order_count,reordered_user_count,product_user_reorder_ratio,product_reorder_ratio,product_user_ratio,product_name,aisle_id,department_id,aisle_reorder_ratio,dept_reorder_ratio
0,1,716,0.350873,1852,1136,276.0,0.385475,0.613391,0.003472,Chocolate Sandwich Cookies,61,19,0.548698,0.574180
1,2,78,0.250289,90,12,8.0,0.102564,0.133333,0.000378,All-Seasons Salt,104,13,0.152391,0.346721
2,3,74,0.350600,277,203,36.0,0.486486,0.732852,0.000359,Robust Golden Unsweetened Oolong Tea,94,7,0.527615,0.653460
3,4,182,0.200119,329,147,64.0,0.351648,0.446809,0.000883,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,0.556655,0.541885
4,5,6,0.193929,15,9,4.0,0.666667,0.600000,0.000029,Green Chile Anytime Sauce,5,13,0.280627,0.346721


In [8]:
user_product = pd.read_csv(datadir+'user_product.csv')
user_product = pd.merge(user_product, products[['product_id', 'weights',
       'product_user_reorder_ratio', 'product_reorder_ratio',
       'product_user_ratio','aisle_reorder_ratio', 'dept_reorder_ratio']], on = 'product_id', how = 'left' )

In [9]:
users = pd.read_csv(datadir +'users_summary.csv')

users.head()

,user_id,total_reordered,order_count,product_count,total_count,reordered_product_count,reordered_order_count,user_product_reorder_ratio,user_order_reorder_ratio,user_total_reorder_ratio,average_product_per_order
0,1,41,10,18,59,10.0,9.0,0.555556,1.000000,0.694915,5.900000
1,2,93,14,102,195,37.0,12.0,0.362745,0.923077,0.476923,13.928571
2,3,55,12,33,88,19.0,11.0,0.575758,1.000000,0.625000,7.333333
3,4,1,5,17,18,1.0,1.0,0.058824,0.250000,0.055556,3.600000
4,5,14,4,23,37,8.0,3.0,0.347826,1.000000,0.378378,9.250000


In [10]:
user_product = pd.merge(user_product, users[['user_id','user_product_reorder_ratio', 'user_order_reorder_ratio',
       'user_total_reorder_ratio', 'average_product_per_order']], on = 'user_id', how = 'left' )

user_product.head()

,user_id,product_id,weights_sum,weights_mean,order_weight_max,order_weight_count,order_weight_sum,time_weight_max,time_weight_sum,days_sum,...,weights,product_user_reorder_ratio,product_reorder_ratio,product_user_ratio,aisle_reorder_ratio,dept_reorder_ratio,user_product_reorder_ratio,user_order_reorder_ratio,user_total_reorder_ratio,average_product_per_order
0,1,196,8.750000,0.875000,0.909091,10,5.000000,0.926316,4.847368,921.0,...,0.530107,0.582500,0.776480,0.038796,0.638832,0.653460,0.555556,1.0,0.694915,5.9
1,1,10258,3.033333,0.337037,0.909091,9,4.909091,0.926316,4.847368,921.0,...,0.417986,0.552962,0.713772,0.002701,0.519170,0.574180,0.555556,1.0,0.694915,5.9
2,1,10326,0.200000,0.200000,0.454545,1,0.454545,0.489474,0.489474,93.0,...,0.418740,0.521581,0.652009,0.009325,0.718104,0.649913,0.555556,1.0,0.694915,5.9
3,1,12427,4.277778,0.427778,0.909091,10,5.000000,0.926316,4.847368,921.0,...,0.416014,0.529482,0.740735,0.008142,0.591986,0.574180,0.555556,1.0,0.694915,5.9
4,1,13032,0.491667,0.163889,0.909091,3,1.727273,0.926316,1.700000,323.0,...,0.357912,0.479782,0.657158,0.006236,0.571584,0.560922,0.555556,1.0,0.694915,5.9


In [11]:
user_product.to_csv(datadir +'user_product_info.csv', index = False)